In [2]:
from pathlib import Path

import pandas as pd

In [3]:
dates_to_visualize = [
    "2019-02",
    "2020-02",
    "2021-02",
    "2022-02",
    "2023-02",
    # "2024-02",
]

In [4]:
available_data_files = list(Path("../data/httparchive_metrics/nel_domain_resource_monitoring_stats").glob("*.parquet"))

used_data_files = list(filter(lambda file: file.stem in dates_to_visualize, available_data_files))
used_data_files

[WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2019-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2020-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2021-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2022-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2023-02.parquet')]

### Load TRANCO Popular domain list


In [5]:
from src.metric_utils import load_tranco_list_from_custom_path

tranco_dict = {}
for month in dates_to_visualize:
    months_list = load_tranco_list_from_custom_path(Path("../resources"), *month.split('-'))
    tranco_dict[month] = months_list

tranco_dict[month]

,order,popular_domain_name
0,1,google.com
1,2,a-msedge.net
2,3,youtube.com
3,4,facebook.com
4,5,microsoft.com
...,...,...
999995,999996,zio.to
999996,999997,tostado.com.ar
999997,999998,comnewsvideo.jp
999998,999999,direweb.it


### Aggregate result to visualize

In [6]:
from results.result_utils import date_to_text_format, concat_data_from_files

result = concat_data_from_files(used_data_files)

result['date_formatted'] = result['date'].map(date_to_text_format)

result

,date,url_domain,url_domain_hosted_resources,url_domain_hosted_resources_with_nel,url_domain_monitored_resources_ratio,date_formatted
0,2019-02,0d38c32709c097507fb7d35fbc48545e.report-uri.com,1,1,100.000000,Feb 2019
1,2019-02,1627f29bce741ebdc46108ecd8ebba3c.report-uri.com,1,1,100.000000,Feb 2019
2,2019-02,18d36df5be4d2f0680090c55b489865d.report-uri.com,1,1,100.000000,Feb 2019
3,2019-02,1e031bc28af67e84e052beae680ccd74.report-uri.com,1,1,100.000000,Feb 2019
4,2019-02,21torr.report-uri.com,2,2,100.000000,Feb 2019
...,...,...,...,...,...,...
1977381,2023-02,zzzw.de,63,62,98.410004,Feb 2023
1977382,2023-02,zzzz.bg,6,6,100.000000,Feb 2023
1977383,2023-02,zzzz.tw,42,42,100.000000,Feb 2023
1977384,2023-02,zzzzap.nl,37,37,100.000000,Feb 2023


### Filter result data to popular collector provider domains

In [10]:
popular_result = pd.DataFrame({})
for month in dates_to_visualize:
    month_result = result[result['date'] == month]
    
    month_result = month_result[month_result['url_domain'].isin(tranco_dict[month]['popular_domain_name'])]
    month_result.reset_index(drop=True, inplace=True)
    
    popular_result = pd.concat([popular_result, month_result])
    
popular_result

,date,url_domain,url_domain_hosted_resources,url_domain_hosted_resources_with_nel,url_domain_monitored_resources_ratio,date_formatted
39706,2022-02,pippio.com,612808,2,0.0,Feb 2022
15637,2023-02,google.com,72623,1,0.0,Feb 2023
39552,2023-02,pippio.com,812682,4,0.0,Feb 2023
59034,2023-02,unpkg.com,41443,2,0.0,Feb 2023


### POPULAR NEL Domain monitored resources ratio distribution

In [11]:
from results.result_utils import get_first_or_0

distribution_result = pd.DataFrame({}, index=[
    '(0%-10%>',
    '(10%-25%>',
    '(25%-50%>',
    '(50%-75%>',
    '(75%-100%)',
    '<100%>',
])

for month in dates_to_visualize:
    current_month_data = popular_result[popular_result['date'] == month].copy()
    
    month_data_col = []

    # month_result['(0%-10%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 0.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 10.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(10%-25%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 10.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 25.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(25%-50%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 25.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 50.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(50%-75%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 50.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 75.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(75%-100%)']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 75.0) & (current_month_data['url_domain_monitored_resources_ratio'] < 100.0)].count()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['<100%>']
    next_val = current_month_data[current_month_data['url_domain_monitored_resources_ratio'] == 100.0].count()
    month_data_col.append(get_first_or_0(next_val))
    
    
    distribution_result[date_to_text_format(month)] = month_data_col
    
distribution_result

,Feb 2019,Feb 2020,Feb 2021,Feb 2022,Feb 2023
(0%-10%>,3,15,94,402,352
(10%-25%>,1,18,48,154,42
(25%-50%>,0,372,356,171,116
(50%-75%>,1,359,615,227,220
(75%-100%),4,316,2353,6281,5462
<100%>,4,4824,53579,52283,59935
